*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you will create a book recommendation algorithm using **K-Nearest Neighbors**.

You will use the [Book-Crossings dataset](http://www2.informatik.uni-freiburg.de/~cziegler/BX/). This dataset contains 1.1 million ratings (scale of 1-10) of 270,000 books by 90,000 users. 

After importing and cleaning the data, use `NearestNeighbors` from `sklearn.neighbors` to develop a model that shows books that are similar to a given book. The Nearest Neighbors algorithm measures distance to determine the “closeness” of instances.

Create a function named `get_recommends` that takes a book title (from the dataset) as an argument and returns a list of 5 similar books with their distances from the book argument.

This code:

`get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")`

should return:

```
[
  'The Queen of the Damned (Vampire Chronicles (Paperback))',
  [
    ['Catch 22', 0.793983519077301], 
    ['The Witching Hour (Lives of the Mayfair Witches)', 0.7448656558990479], 
    ['Interview with the Vampire', 0.7345068454742432],
    ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.5376338362693787],
    ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.5178412199020386]
  ]
]
```

Notice that the data returned from `get_recommends()` is a list. The first element in the list is the book title passed in to the function. The second element in the list is a list of five more lists. Each of the five lists contains a recommended book and the distance from the recommended book to the book passed in to the function.

If you graph the dataset (optional), you will notice that most books are not rated frequently. To ensure statistical significance, remove from the dataset users with less than 200 ratings and books with less than 100 ratings.

The first three cells import libraries you may need and the data to use. The final cell is for testing. Write all your code in between those cells.

Do NOT remove the 0 values.
Remove users that appear <200 times in the list and books that have <100 users from df_ratings (using the isbn column for the latter). This should be done at the same time (i.e. you can’t just remove users first then books from the resulting dataset).
Now merge df_ratings with df_books.
Drop the title duplicates with the default keep='first' parameter setting (so no need to get the max or the mean rating for the duplicated bunch) when using the drop_duplicates function.
For your final answer, the recommendations sublist should be in backwards order for some reason to pass.

https://forum.freecodecamp.org/t/book-recommendation-knn-system/412976/6

In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
from collections import Counter

In [2]:
# get data files
# !wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
# !unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title'],
    usecols=['isbn', 'title'],
    dtype={'isbn': 'str', 'title': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [4]:
# Removes all duplicates from the list of book titles and ISBNs
df_books = df_books.drop_duplicates(keep='first')

i_min = 100 # minimum number of times a book must have been rated to be included
u_min = 200 # minimum number of times a user must have rated a book to be included

# Removes all entries from the dataset based on the above perameters
df_ratings = df_ratings[
    df_ratings['isbn'].isin(
    {k: v for (k, v) in Counter(df_ratings.isbn).items() if v >= i_min}.keys()) &
    df_ratings['user'].isin(
    {k: v for (k, v) in Counter(df_ratings.user).items() if v >= u_min}.keys())
]

# Merges list of books and ratings into a finished data set
df = pd.merge(df_ratings, df_books, on=['isbn'])

In [5]:
# Generates a reference list of all books in the finished dataset 
ref = df[['isbn','title']].drop_duplicates()
# ISBN : Title conversion tools
TI = dict(zip(ref.title, ref.isbn)) # Returns an ISBN when given a title TI["title"]
IT = {v: k for k, v in TI.items()} # Returns an title when given an ISBN IT["ISBN"]

In [6]:
# Makes pivot of finished data set
pivot = df.pivot('isbn', 'user', 'rating').fillna(0)
# KNN model
KNN = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
# fits model to data
KNN.fit(pivot.values)

NearestNeighbors(algorithm='brute', metric='cosine')

In [9]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
    distance, indices = KNN.kneighbors(
        # distance from target
        np.array(pivot.T[TI[str(book)]]).reshape(1,-1),
        # number of recomendations
        n_neighbors=4)
    # Generates list of recomendations
    recommends = [
        (IT[pivot.index[indices.flatten()[i]]], distance.flatten()[i]) 
        for i in range(1, len(distance.flatten()))
    ]
    # formats the list of recomentations to spesifications
    recommends.reverse()
    # returns final formated results
    return [book,recommends]

Use the cell below to test your function. The `test_book_recommendation()` function will inform you if you passed the challenge or need to keep trying.

In [10]:
def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

You passed the challenge! 🎉🎉🎉🎉🎉
